In [9]:
import myfunctions as mf
from skimage.io import imshow
import napari 

The code with either 4D filtering or napari viewer can only be run on the cluster.

In [2]:
# mf.segment4D(exp=mf.exp_list()[0], filtering4D=False)


Exp P28A_FT_H_Exp1 segmentation started

Loading first volume...

Finding threshold...
Threshold=1.50 found in 0.35 s



In [6]:
exp = mf.exp_list()[0]
first_volume = mf.load_volume(exp=exp, time=140, isImage=True, OS='Windows')

In [10]:
viewer = napari.Viewer()
viewer.add_image(first_volume, name='first volume')

<Image layer 'first volume' at 0x1ee2aadb040>